In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import inegi_key #inegi token
from config import coneval_key #coneval
from config import gkey #google key

In [2]:


url = "https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr"
#declare parameters

EntFed = "15" #entidad federativa
Mpo    = "0" #municipio dentro de la entidad federativa
Loc    = "0" #loc dentro del mpo
AGEB   = "0" #AGEB no sé que es AGEB
Mnz    = "0" #manzana tres dígitos
Sector = "0" # Sector de la actividad económica (ej. 46 )
Subsec = "0" # Subsector de la actividad económica (ej. 464 )
Rama   = "0" #Rama de la actividad económica (ej. 4641)
Clase  = "0" #Clase de la actividad económica (ej. 464112)
Nomest = "0" # Nombre del establecimiento
Regini = "1" #Número de registro a partir del cuál se mostrarán los resultados de la búsqueda. 
Regfin = "25" #Número de registro final que se mostrará en resultados de la búsqueda. 
Id     = "0" #Clave única del establecimiento.
Estr   = "0" # Clave de un dígito del estrato. Para incluir todos los tamaños se especifica 0.
                #1. Para incluir de 0 a 5 personas.
                #2. Para incluir de 6 a 10 personas.
                #3. Para incluir de 11 a 30 personas.
                #4. Para incluir de 31 a 50 personas.
                #5. Para incluir de 51 a 100 personas.
                #6. Para incluir de 101 a 250 personas.
                #7. Para incluir de 251 y más person

#Ejemplo
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[Aquí va tu token]
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[8455939b-9726-bb8c-c70d-681609b40110]
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[5537bc61-be6f-dddf-a592-e3cd4417e515]


urlconsulta = url + "/" + EntFed + "/" + Mpo + "/" + Loc + "/" + AGEB + "/" + Mnz + "/" + Sector + "/" + Subsec + "/" + Rama + "/" + Clase + "/" + Nomest + "/" + Regini + "/" + Regfin + "/" + Id + "/" + Estr + "/" + "[" + inegi_key + "]"

print(urlconsulta)
print(inegi_key)

https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/15/0/0/0/0/0/0/0/0/0/1/25/0/0/[8455939b-9726-bb8c-c70d-681609b40110]
8455939b-9726-bb8c-c70d-681609b40110


In [ ]:
denue_df = pd.read_csv("Resources/DENUE_full_data.csv",encoding="ISO-8859-1")
#denue_df = pd.read_csv("Resources/DENUE_full_data_old.csv",encoding="ISO-8859-1")
denue_df

In [ ]:
list(denue_df)

In [ ]:
denue_df = denue_df[['Código de la clase de actividad SCIAN',
                     'Nombre de clase de la actividad',
                     'Nombre de la Unidad Económica',             
                     'Descripcion estrato personal ocupado',
                     'Tipo de asentamiento humano',
                     'Nombre de asentamiento humano',
                     'Código Postal',
                     'Clave entidad',
                     'Entidad federativa',
                     'Clave municipio',
                     'Municipio',
                     'Clave localidad',
                     'Localidad',
                     'Área geoestadística básica ',
                     'Tipo de establecimiento',
                     'Latitud',
                     'Longitud',
                     'Fecha de incorporación al DENUE']]

In [13]:
denue_df["Código de la clase de actividad SCIAN"].astype(str).str[:2].astype(int)
denue_df.insert(1, 'Código Tipo de Industria', denue_df["Código de la clase de actividad SCIAN"].astype(str).str[:2].astype(int))
denue_df

,Código de la clase de actividad SCIAN,Código Tipo de Industria,Nombre de clase de la actividad,Nombre de la Unidad Económica,Descripcion estrato personal ocupado,Tipo de asentamiento humano,Nombre de asentamiento humano,Código Postal,Clave entidad,Entidad federativa,Clave municipio,Municipio,Clave localidad,Localidad,Área geoestadística básica,Tipo de establecimiento,Latitud,Longitud,Fecha de incorporación al DENUE
0,115113,11,Beneficio de productos agrÃ­colas,AGRÃCOLA MARICRUZ,0 a 5 personas,PUEBLO,SAN MARTIN,55850,15,MÃXICO,75,SAN MARTÃN DE LAS PIRÃMIDES,1,SAN MARTÃN DE LAS PIRÃMIDES,154,Fijo,19.703800,-98.836521,jul-10
1,112512,11,"Piscicultura y otra acuicultura, excepto camar...",AGUA ZARCA,0 a 5 personas,PUEBLO,SAN MATEO SEGUNDA SECCIÃN,0,15,MÃXICO,7,AMANALCO,15,SAN MATEO,50,Fijo,19.280833,-100.015000,dic-14
2,112512,11,"Piscicultura y otra acuicultura, excepto camar...",ASOCIACIÃN DE PESCADORES LA CUENCA,6 a 10 personas,COLONIA,CAMINO A CORRAL DE PIEDRA,0,15,MÃXICO,7,AMANALCO,1,AMANALCO DE BECERRA,46,Fijo,19.256304,-100.015107,jul-10
3,112512,11,"Piscicultura y otra acuicultura, excepto camar...",ASOCIACIÃN RURAL DE INTERES COLECTIVO CPB CAÃ...,11 a 30 personas,PARAJE,LA CAÃADA,52769,15,MÃXICO,37,HUIXQUILUCAN,43,LA CAÃADA,168,Fijo,19.319444,-99.372500,jul-10
4,112512,11,"Piscicultura y otra acuicultura, excepto camar...",BARRANCA FRESCA,0 a 5 personas,COLONIA,BARRANCA FRESCA,51200,15,MÃXICO,110,VALLE DE BRAVO,117,BARRANCA FRESCA,016A,Fijo,19.131725,-100.118014,dic-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605580,931610,93,Actividades administrativas de instituciones d...,ZONA ESCOLAR J138,0 a 5 personas,COLONIA,CENTRO,52400,15,MÃXICO,88,TENANCINGO,1,TENANCINGO DE DEGOLLADO,63,Fijo,18.963403,-99.591564,jul-10
605581,931610,93,Actividades administrativas de instituciones d...,ZONA ESCOLAR NO 69,0 a 5 personas,PUEBLO,SAN LORENZO CHIMALPA,56625,15,MÃXICO,25,CHALCO,16,SAN MATEO HUITZILZINGO,1112,Fijo,19.235759,-98.919415,jul-10
605582,931610,93,Actividades administrativas de instituciones d...,ZONA ESCOLAR NÃMERO 35,0 a 5 personas,COLONIA,CENTRO,50450,15,MÃXICO,14,ATLACOMULCO,1,ATLACOMULCO DE FABELA,322,Fijo,19.801127,-99.878191,dic-14
605583,931610,93,Actividades administrativas de instituciones d...,ZONA S040 SUPERVISIÃN ESCOLAR DE ESCUELAS SEC...,0 a 5 personas,COLONIA,SAN ILDEFONSO,54470,15,MÃXICO,60,NICOLÃS ROMERO,1,CIUDAD NICOLÃS ROMERO,019A,Fijo,19.617800,-99.298411,jul-10


In [14]:
denue_df.to_csv("Output/DENUE_Filtered.csv", index=True, header=True)

In [15]:
zip_df = pd.read_csv("Resources/Codigospostales.csv",encoding="ISO-8859-1")
zip_df = zip_df.iloc[0:8127,:]
zip_df

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,d_estado,d_ciudad,d_CP,c_estado,c_oficina,c_CP,c_tipo_asenta,c_mnpio,id_asenta_cpcons,d_zona,c_cve_ciudad
0,50000.0,Centro,Colonia,Toluca,México,Toluca de Lerdo,50091.0,15.0,50091.0,NaN,9.0,106.0,160.0,Urbano,20.0
1,50010.0,Celanese,Colonia,Toluca,México,Toluca de Lerdo,50091.0,15.0,50091.0,NaN,9.0,106.0,163.0,Urbano,20.0
2,50010.0,Club Jardín,Colonia,Toluca,México,Toluca de Lerdo,50091.0,15.0,50091.0,NaN,9.0,106.0,164.0,Urbano,20.0
3,50010.0,Guadalupe,Colonia,Toluca,México,Toluca de Lerdo,50091.0,15.0,50091.0,NaN,9.0,106.0,165.0,Urbano,20.0
4,50010.0,La Cruz Comalco,Colonia,Toluca,México,Toluca de Lerdo,50091.0,15.0,50091.0,NaN,9.0,106.0,166.0,Urbano,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8122,57910.0,Constitución de 1857,Colonia,Nezahualcóyotl,México,Ciudad Nezahualcóyotl,57002.0,15.0,57002.0,NaN,9.0,58.0,4641.0,Urbano,10.0
8123,57920.0,Santa Martha,Colonia,Nezahualcóyotl,México,Ciudad Nezahualcóyotl,57002.0,15.0,57002.0,NaN,9.0,58.0,4642.0,Urbano,10.0
8124,57930.0,Manantiales,Colonia,Nezahualcóyotl,México,Ciudad Nezahualcóyotl,57002.0,15.0,57002.0,NaN,9.0,58.0,4643.0,Urbano,10.0
8125,57940.0,Loma Bonita,Colonia,Nezahualcóyotl,México,Ciudad Nezahualcóyotl,57002.0,15.0,57002.0,NaN,9.0,58.0,4645.0,Urbano,10.0


In [17]:
list(zip_df["d_tipo_asenta"].unique())

['Colonia',
 'Fraccionamiento',
 'Barrio',
 'Unidad habitacional',
 'Zona industrial',
 'Residencial',
 'Pueblo',
 'Gran usuario',
 'Zona federal',
 'Conjunto habitacional',
 'Ejido',
 'Aeropuerto',
 'Ranchería',
 'Rancho',
 'Parque industrial',
 'Paraje',
 'Hacienda',
 'Equipamiento',
 'Exhacienda',
 'Condominio',
 'Zona comercial',
 'Campamento',
 'Granja',
 'Villa',
 'Zona militar']

In [18]:
list(denue_df["Tipo de asentamiento humano"].unique())

['PUEBLO',
 'COLONIA',
 'PARAJE',
 'BARRIO',
 'MANZANA',
 nan,
 'FRACCIONAMIENTO',
 'EJIDO',
 'RANCHERÃ\x8dA',
 'SECTOR',
 'CONDOMINIO',
 'FRACCIÃ\x93N',
 'AMPLIACIÃ\x93N',
 'GRANJA',
 'COTO',
 'PROLONGACIÃ\x93N',
 'PRIVADA',
 'CONJUNTO HABITACIONAL',
 'UNIDAD HABITACIONAL',
 'UNIDAD',
 'SECCIÃ\x93N',
 'ZONA INDUSTRIAL',
 'VILLA',
 'PARQUE INDUSTRIAL',
 'CIUDAD',
 'RANCHO',
 'CUARTEL',
 'CIUDAD INDUSTRIAL',
 'AEROPUERTO',
 'RESIDENCIAL',
 'HACIENDA',
 'REGIÃ\x93N',
 'EXHACIENDA',
 'PUERTO',
 'CANTÃ\x93N',
 'ZONA MILITAR',
 'RINCONADA',
 'ZONA NAVAL',
 'CORREDOR INDUSTRIAL',
 'ZONA FEDERAL',
 'SUPERMANZANA']

In [19]:
###########################################################################################

grouped_denue_df = denue_df.groupby('Clave municipio')
grouped_denue_df.head()

,Código de la clase de actividad SCIAN,Código Tipo de Industria,Nombre de clase de la actividad,Nombre de la Unidad Económica,Descripcion estrato personal ocupado,Tipo de asentamiento humano,Nombre de asentamiento humano,Código Postal,Clave entidad,Entidad federativa,Clave municipio,Municipio,Clave localidad,Localidad,Área geoestadística básica,Tipo de establecimiento,Latitud,Longitud,Fecha de incorporación al DENUE
0,115113,11,Beneficio de productos agrÃ­colas,AGRÃCOLA MARICRUZ,0 a 5 personas,PUEBLO,SAN MARTIN,55850,15,MÃXICO,75,SAN MARTÃN DE LAS PIRÃMIDES,1,SAN MARTÃN DE LAS PIRÃMIDES,154,Fijo,19.703800,-98.836521,jul-10
1,112512,11,"Piscicultura y otra acuicultura, excepto camar...",AGUA ZARCA,0 a 5 personas,PUEBLO,SAN MATEO SEGUNDA SECCIÃN,0,15,MÃXICO,7,AMANALCO,15,SAN MATEO,50,Fijo,19.280833,-100.015000,dic-14
2,112512,11,"Piscicultura y otra acuicultura, excepto camar...",ASOCIACIÃN DE PESCADORES LA CUENCA,6 a 10 personas,COLONIA,CAMINO A CORRAL DE PIEDRA,0,15,MÃXICO,7,AMANALCO,1,AMANALCO DE BECERRA,46,Fijo,19.256304,-100.015107,jul-10
3,112512,11,"Piscicultura y otra acuicultura, excepto camar...",ASOCIACIÃN RURAL DE INTERES COLECTIVO CPB CAÃ...,11 a 30 personas,PARAJE,LA CAÃADA,52769,15,MÃXICO,37,HUIXQUILUCAN,43,LA CAÃADA,168,Fijo,19.319444,-99.372500,jul-10
4,112512,11,"Piscicultura y otra acuicultura, excepto camar...",BARRANCA FRESCA,0 a 5 personas,COLONIA,BARRANCA FRESCA,51200,15,MÃXICO,110,VALLE DE BRAVO,117,BARRANCA FRESCA,016A,Fijo,19.131725,-100.118014,dic-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17856,312112,31,PurificaciÃ³n y embotellado de agua,PURIFICADORA DE AGUA SAN PURA,0 a 5 personas,BARRIO,SAN JOSÃ,51470,15,MÃXICO,77,SAN SIMÃN DE GUERRERO,1,SAN SIMÃN DE GUERRERO,36,Fijo,19.022881,-100.006338,dic-14
23501,311830,31,ElaboraciÃ³n de tortillas de maÃ­z y molienda ...,TORTILLERÃA AYAPANGO,0 a 5 personas,BARRIO,DE LA SOLEDAD,56760,15,MÃXICO,17,AYAPANGO,1,AYAPANGO DE GABRIEL RAMOS M.,23,Fijo,19.126029,-98.804848,jul-10
23538,311830,31,ElaboraciÃ³n de tortillas de maÃ­z y molienda ...,TORTILLERÃA BARRUETA,0 a 5 personas,BARRIO,SANTA ROSA,51407,15,MÃXICO,77,SAN SIMÃN DE GUERRERO,1,SAN SIMÃN DE GUERRERO,36,Fijo,19.022677,-100.007783,jul-10
28998,311830,31,ElaboraciÃ³n de tortillas de maÃ­z y molienda ...,TORTILLERÃA OCAMPO,0 a 5 personas,BARRIO,BARRIO CRUZ VERDE,51470,15,MÃXICO,77,SAN SIMÃN DE GUERRERO,1,SAN SIMÃN DE GUERRERO,36,Fijo,19.022722,-100.006450,dic-14


In [20]:
# contar establecimientos por municipios
count_est = denue_df["Clave municipio"].value_counts()
count_est.head()

33     71819
58     48815
106    45343
57     28969
104    27111
Name: Clave municipio, dtype: int64

In [21]:
count_industria = grouped_denue_df["Código Tipo de Industria"].value_counts()
count_industria.head(30)

Clave municipio  Código Tipo de Industria
1                46                           527
                 31                           146
                 81                           108
                 33                            99
                 72                            93
                 93                            69
                 62                            41
                 61                            33
                 56                            29
                 32                            25
                 43                            21
                 53                            11
                 54                            10
                 71                            10
                 52                             6
                 11                             5
                 51                             4
                 23                             2
                 48                             2
        

In [22]:
count_numempl = grouped_denue_df["Descripcion estrato personal ocupado"].value_counts()
count_numempl.head(25)

Clave municipio  Descripcion estrato personal ocupado
1                0 a 5 personas                          1159
                 6 a 10 personas                           40
                 11 a 30 personas                          28
                 51 a 100 personas                          7
                 31 a 50 personas                           6
                 101 a 250 personas                         1
                 251 y mÃ¡s personas                        1
2                0 a 5 personas                          5895
                 6 a 10 personas                          172
                 11 a 30 personas                         122
                 31 a 50 personas                          20
                 101 a 250 personas                         6
                 251 y mÃ¡s personas                        4
                 51 a 100 personas                          4
3                0 a 5 personas                           639
                

In [40]:
denue_df_sample = denue_df.sample(frac=.12)

In [37]:
denue_df_sample.loc[denue_df_sample["Descripcion estrato personal ocupado"].isin(['11 a 30 personas']),'nombres']='T'
display(denue_df_sample.head())
ddd=denue_df_sample.loc[denue_df_sample["Descripcion estrato personal ocupado"]=='11 a 30 personas']
display(ddd.head())

,Código de la clase de actividad SCIAN,Código Tipo de Industria,Nombre de clase de la actividad,Nombre de la Unidad Económica,Descripcion estrato personal ocupado,Tipo de asentamiento humano,Nombre de asentamiento humano,Código Postal,Clave entidad,Entidad federativa,Clave municipio,Municipio,Clave localidad,Localidad,Área geoestadística básica,Tipo de establecimiento,Latitud,Longitud,Fecha de incorporación al DENUE,nombres
361954,461121,46,Comercio al por menor de carnes rojas,VISCERAS CRUDAS PRECOSIDAS Y CÃRNICOS SEMICO...,0 a 5 personas,COLONIA,TECAMAC CENTRO,55740,15,MÃXICO,81,TECÃMAC,1,TECÃMAC DE FELIPE VILLANUEVA,325,Fijo,19.711787,-98.972871,jul-10,NaN
303889,461110,46,"Comercio al por menor en tiendas de abarrotes,...",TIENDA ANTONI,0 a 5 personas,COLONIA,JARDINES DE CHALCO,56600,15,MÃXICO,25,CHALCO,1,CHALCO DE DÃAZ COVARRUBIAS,081A,Fijo,19.259209,-98.911846,dic-14,NaN
246435,465311,46,Comercio al por menor de artÃ­culos de papelerÃ­a,PAPELERÃA LA MICHOACANA,0 a 5 personas,COLONIA,MIRAMAR,56343,15,MÃXICO,31,CHIMALHUACÃN,1,CHIMALHUACÃN,662,Fijo,19.382576,-98.963425,dic-14,NaN
21653,311830,31,ElaboraciÃ³n de tortillas de maÃ­z y molienda ...,TORTILLAS DE COMAL LA CHIQUITA,0 a 5 personas,BARRIO,CANTEROS,56357,15,MÃXICO,31,CHIMALHUACÃN,1,CHIMALHUACÃN,054A,Fijo,19.418607,-98.977898,dic-14,NaN
185567,461110,46,"Comercio al por menor en tiendas de abarrotes,...",MISCELÃNEA ALEJANDRO,0 a 5 personas,COLONIA,VALLE DE LOS REYES,56430,15,MÃXICO,70,LA PAZ,1,LOS REYES ACAQUILPAN,143,Fijo,19.365144,-98.978913,jul-10,NaN


,Código de la clase de actividad SCIAN,Código Tipo de Industria,Nombre de clase de la actividad,Nombre de la Unidad Económica,Descripcion estrato personal ocupado,Tipo de asentamiento humano,Nombre de asentamiento humano,Código Postal,Clave entidad,Entidad federativa,Clave municipio,Municipio,Clave localidad,Localidad,Área geoestadística básica,Tipo de establecimiento,Latitud,Longitud,Fecha de incorporación al DENUE,nombres
406268,611121,61,Escuelas de educaciÃ³n primaria del sector pri...,ESCUELA AMERICAN HILLS,11 a 30 personas,COLONIA,MARAVILLAS,57410,15,MÃXICO,58,NEZAHUALCÃYOTL,1,CIUDAD NEZAHUALCÃYOTL,685,Fijo,19.419224,-99.046359,jul-10,T
440412,621116,62,ClÃ­nicas de consultorios mÃ©dicos del sector ...,UNIDAD DE MEDICINA FAMILIAR N 80,11 a 30 personas,COLONIA,VISTA HERMOSA,54001,15,MÃXICO,104,TLALNEPANTLA DE BAZ,1,TLALNEPANTLA,904,Fijo,19.516714,-99.227211,jul-10,T
419083,611132,61,Escuelas de educaciÃ³n secundaria general del ...,SECUNDARIA NO 0048 LICENCIADO ADOLFO LOPEZ MATEOS,11 a 30 personas,PUEBLO,SAN MATEO IXTACALCO,54840,15,MÃXICO,121,CUAUTITLÃN IZCALLI,1,CUAUTITLÃN IZCALLI,89,Fijo,19.704255,-99.175970,jul-10,T
362308,463310,46,Comercio al por menor de calzado,VIVA ZAPATOS,11 a 30 personas,COLONIA,VALLE DE LAS FLORES,55700,15,MÃXICO,20,COACALCO DE BERRIOZÃBAL,1,SAN FRANCISCO COACALCO,160,Fijo,19.633875,-99.096383,jul-10,T
600316,931210,93,AdministraciÃ³n pÃºblica en general,DIRECCIÃN DE GOBIERNO,11 a 30 personas,COLONIA,CENTRO,50000,15,MÃXICO,106,TOLUCA,1,TOLUCA DE LERDO,453,Fijo,19.290976,-99.656699,jul-10,T


In [41]:
denue_df_sample.to_csv("Output/DENUE_Filtered_complete.csv", index=True, header=True)

In [43]:
denue_df_sample

,ID,Nombre de la Unidad Económica,Razón social,Código de la clase de actividad SCIAN,Nombre de clase de la actividad,Descripcion estrato personal ocupado,Tipo de vialidad,Nombre de la vialidad,Tipo de entre vialidad 1,Nombre de entre vialidad 1,...,Localidad,Área geoestadística básica,Manzana,NÃºmero de telÃ©fono,Correo electrÃ³nico,Sitio en Internet,Tipo de establecimiento,Latitud,Longitud,Fecha de incorporación al DENUE
485138,2014798,MICHELADAS 6 GRADOS,NaN,722412,"Bares, cantinas y similares",6 a 10 personas,CALLE,AVENIDA VALLE DEL YANG-TSE,CALLE,VALLE DEL COLORADO,...,CIUDAD NEZAHUALCÃYOTL,30,6,NaN,NaN,NaN,Fijo,19.492879,-99.059727,jul-10
180862,6354673,MEGA COMERCIAL MEXICANA ALAMEDAS SUC 23,TIENDAS COMERCIAL MEXICANA S.A. DE C.V.,462111,Comercio al por menor en supermercados,101 a 250 personas,CALLE,ADOLFO LÃPEZ MATEOS,CALLE,DE LOS CISNES,...,CIUDAD LÃPEZ MATEOS,202,17,NaN,MMUNOZ@COMERCI.COM.MX,WWW.COMERCIALMEXICANA.COM.MX,Fijo,19.552485,-99.241193,jul-10
311724,2363453,TIENDA DE ABARROTES SIN NOMBRE,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,AVENIDA,JUÃREZ,CALLE,VISTA HERMOSA,...,RÃO FRÃO DE JUÃREZ,834,2,NaN,NaN,NaN,Fijo,19.355595,-98.672878,dic-14
52732,2343091,MAQUINAS PARA SERIGRAFÃA,NaN,333245,FabricaciÃ³n de maquinaria y equipo para la in...,0 a 5 personas,CALLE,MAYAQUEN,AVENIDA,ORGANIZACIÃN POPULAR,...,CHIMALHUACÃN,008A,58,5.522284e+09,NaN,NaN,Fijo,19.432340,-98.974542,dic-14
452328,2020353,ANTOJITOS MEXICANOS SIN NOMBRE,NaN,722513,Restaurantes con servicio de preparaciÃ³n de a...,0 a 5 personas,CALLE,GUANAJUATO,CALLE,VERACRUZ,...,ECATEPEC DE MORELOS,320A,17,NaN,NaN,NaN,Fijo,19.507917,-99.028781,jul-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287589,2451555,RECAUDERÃA SIN NOMBRE,NaN,461130,Comercio al por menor de frutas y verduras fre...,0 a 5 personas,CALZADA,DEL PANTEÃN,PRIVADA,NINGUNO,...,SAN MATEO ATENCO,359,15,NaN,NaN,NaN,Fijo,19.267672,-99.537364,dic-14
550479,2187493,LA IGLESIA DE JESUCRISTO DE LOS SANTOS DE LOS ...,LA IGLESIA DE JESUCRISTO DE LOS SANTOS DE LOS ...,813210,Asociaciones y organizaciones religiosas,101 a 250 personas,AVENIDA,TEXCOCO,CALLE,AVENIDA DE LAS TORRES,...,CIUDAD NEZAHUALCÃYOTL,1310,13,NaN,NaN,NaN,Fijo,19.377524,-99.008693,jul-10
88996,2077986,ABARROTES Y CREMERÃA CECI,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,VALLE PERDIDO,ANDADOR,4,...,ECATEPEC DE MORELOS,2860,25,5.557129e+09,NaN,NaN,Fijo,19.507421,-99.043806,jul-10
226940,2154985,MUEBLERÃA LA RIVERA DE VERACRUZ,NaN,466111,Comercio al por menor de muebles para el hogar,0 a 5 personas,AVENIDA,CUAUHTÃMOC,CALLE,JACARANDAS,...,IXTAPALUCA,1071,14,5.559763e+09,NaN,NaN,Fijo,19.309784,-98.913128,jul-10
